<a href="https://colab.research.google.com/github/saided/pandas_using_python/blob/master/grp_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import re
import requests
import xlrd
import openpyxl
from collections import ChainMap
from requests import Timeout

In [3]:
def root_group():
    url = "https://api.cloud.jiosurveillance.com/auth/oauth/token"
    username = input("Enter account ID")
    #print(username)
    password = input("Enter account password")
    #print(password)

    payload = 'client_secret=8b078e3496893ae3b9d7ccb83a6befdb&client_id=firmware-upgrade-job&grant_type=password&username={}&password={}'.format(
    username, password)
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
            }
    try:
        response = requests.request("POST", url, headers=headers, data=payload, timeout=6)
        #print(response.text.encode('utf8'))
        #print(response.json())
    except: Timeout

    r = response.json()
    access_token = r['access_token']
    token = str(access_token)
    #print(token)
    own_id = r['owner_id']
    own_id = str(own_id)
    #print(own_id)

    url = "https://api.cloud.jiosurveillance.com/folders?op=FIND"

    payload = "{\n\t\"user\" : \"%s\"\n}" % (own_id)
    headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % (token)
    }
    #print(payload)
    response = requests.request("POST", url, headers=headers, data=payload, timeout=6)
    #print(response.text.encode('utf8')) # it gives total info about groups present in that  account
    #print(response.url)

    own_response = response.json()
    print(own_response)
    b = own_response['result']
    #print(b)
    c = b['items']
    #print(c[0])
    #print(len(c))
    #print('Total no.of existing groups in the account :',len(c)-1)
    mast_grp_id = c[0]['id']
    #print(mast_grp_id,type(mast_grp_id))
    if len(c)==0:
      print('something went wrong')
    return(mast_grp_id,own_id,token)

print(root_group())  

Enter account IDrr.demo@gmail.com
Enter account passwordJioadmin123
{'success': True, 'result': {'items': [{'id': '801-9uTasQr94a', 'name': '__root__', 'subfolders': [], 'cameras': [], 'owner_id': '801000011026', 'owner_name': 'rr.demo@gmail.com', 'root': True, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}]}}
('801-9uTasQr94a', '801000011026', '801-Ubf2d3485-1e43-4eeb-9013-46fcd60bfe78')


In [39]:
is_root_grp = list(root_group())
print(len(is_root_grp))
mast_grp_id = is_root_grp[0]
own_id = is_root_grp[1]
token = is_root_grp[2]
#print(token,type(token))

Enter account IDrr.demo@gmail.com
Enter account passwordJioadmin123
{'success': True, 'result': {'items': [{'id': '801-9uTasQr94a', 'name': '__root__', 'subfolders': ['801-wud0JvV70D', '801-Aby22GXjs7'], 'cameras': [], 'owner_id': '801000011026', 'owner_name': 'rr.demo@gmail.com', 'root': True, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}, {'id': '801-wud0JvV70D', 'name': 'NORTH', 'subfolders': ['801-TliYIdSFJ7', '801-p8QMTywVJG', '801-ssBSoIMOUh', '801-pAtXjOPxFH', '801-tMtlGMxHsc', '801-RYOSM5y6OR'], 'cameras': [], 'owner_id': '801000011026', 'owner_name': 'rr.demo@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}, {'id': '801-Aby22GXjs7', 'name': 'SOUTH', 'subfolders': ['801-VxAeAHciml', '801-nE72QWPPjz', '801-4yS4swq6vJ'], 'cameras': [], 'owner_id': '801000011026', 'owner_name': 'rr.demo@gmail.com', 'root': False, 'per

In [5]:

def create_zone(cellvalue,grp_id,token):
      #cellvalue1 = list(cellvalue)
      #if cellvalue not in cellvalue1:
        url = "https://api.cloud.jiosurveillance.com/folders?op=CREATE"
        payload = '{\n\t\"parent_folder\" : \"' + grp_id + '\",\n\t\"name\" : \"' + cellvalue + "\"\n}"
        print(payload)
        headers = {
          'Content-Type': 'application/json',
          'Authorization': 'Bearer %s' % token
        }

        response = requests.request("POST", url, headers=headers, data=payload, timeout=6)
        r = response.json()
        return r
        #cellvalue1.add(cellvalue)




In [28]:
North=0
South=0
theFile = openpyxl.load_workbook('/content/Tree structure.xlsx')
allSheetNames = theFile.sheetnames
#print(allSheetNames)
print("All sheet names {} " .format(allSheetNames))
for sh in xlrd.open_workbook('/content/Tree structure.xlsx').sheets():
  print('Total no of rows',sh.nrows)
  print('Total no of columns',sh.ncols)
  zone_folder = []
  #print(North)
  for row in range(1, sh.nrows):
    #print(row)
    #cellcontent=[]
    for col in range(sh.ncols):
      mycell = sh.cell(row,col).value
      #print(mycell)
      #cellcontent.append(mycell)
      #dict[row]= cellcontent
    #print(cellcontent)
    #print(dict)
      if (len(is_root_grp) !=0 and col==6):
        print(col,mycell)
        if (mycell=='NORTH' and North==0):
          zone_folder.append(create_zone(mycell,mast_grp_id,token))
          North+=1
        elif (mycell=='SOUTH' and South==0):
          zone_folder.append(create_zone(mycell,mast_grp_id,token))
          South+=1
        else:
           print("Repeated")
           break
      #print(n)




All sheet names ['Sheet1'] 
Total no of rows 192
Total no of columns 12
6 WEST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 WEST
Repeated
6 WEST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 NORTH
{
	"parent_folder" : "801-9uTasQr94a",
	"name" : "NORTH"
}
6 SOUTH
{
	"parent_folder" : "801-9uTasQr94a",
	"name" : "SOUTH"
}
6 WEST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 WEST
Repeated
6 WEST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 NORTH
Repeated
6 NORTH
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 NORTH
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 EAST
Repeated
6 NORTH
Repeated
6 EAST
Repeated
6 SOUTH
Repeated
6 NORTH
Repeated
6 NORTH
Repeated
6 NORTH
Repeated
6 NORTH
Repeated
6 NORTH
Repeated
6 NORTH
Repeated
6 NORTH
Repeated
6 NORTH
Repeated
6 NORTH
Repeated
6 NORTH
Repeated
6 NORTH
Repeated
6 NORTH
Repeated
6 NORTH
R

In [29]:
print((zone_folder))
zone_id_1 = zone_folder[0]['result']['id']
zone_id_2 = zone_folder[1]['result']['id']

print(zone_id_1,zone_id_2)

[{'success': True, 'result': {'id': '801-wud0JvV70D', 'name': 'NORTH', 'subfolders': [], 'cameras': [], 'owner_id': '801000011026', 'owner_name': 'rr.demo@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}, {'success': True, 'result': {'id': '801-Aby22GXjs7', 'name': 'SOUTH', 'subfolders': [], 'cameras': [], 'owner_id': '801000011026', 'owner_name': 'rr.demo@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}]
801-wud0JvV70D 801-Aby22GXjs7


In [30]:

def create_substate(statevalue,zone_id,token):
      #cellvalue1 = list(cellvalue)
      #if cellvalue not in cellvalue1:
        url = "https://api.cloud.jiosurveillance.com/folders?op=CREATE"
        payload = '{\n\t\"parent_folder\" : \"' + zone_id + '\",\n\t\"name\" : \"' + statevalue + "\"\n}"
        print(payload)
        headers = {
          'Content-Type': 'application/json',
          'Authorization': 'Bearer %s' % token
        }

        response = requests.request("POST", url, headers=headers, data=payload, timeout=6)
        r = response.json()
        return r
        #cellvalue1.add(cellvalue)




In [31]:
#print(n)
#zone_id = n['result']['id']
#print(zone_id)
for sh in xlrd.open_workbook('/content/Tree structure.xlsx').sheets():
  northList = []
  southList = []
  north_folder = []
  south_folder = []
  for row in range(1, sh.nrows):
    for col in range(1, 7):
      mycell = sh.cell(row,col).value
      if mycell=='NORTH':
        statecell = sh.cell(row, col+1).value
        #statecell.value = str(statecell.value)
        if statecell.upper() in [name.upper() for name in northList]:
          break
        else:
          northList.append(statecell)
          print("NorthList contains following list" + str(northList))
          north_folder.append(create_substate(statecell,zone_id_1,token))
      elif mycell == 'SOUTH':
          statecell = sh.cell(row,col+1).value
          if statecell.upper() in [name.upper() for name in southList]:
            break
          else:
            southList.append(statecell)
            print("SouthList contains following list" + str(southList))
            south_folder.append(create_substate(statecell,zone_id_2,token))
          #print("Received folder id for NORTH is" + folder_id[0])
          #check_state_present = check_state_created(cell_content, folder_id)
          #create_folder(folder_id[0], stateCell.value, token)

NorthList contains following list['JAMMU KASHMIR']
{
	"parent_folder" : "801-wud0JvV70D",
	"name" : "JAMMU KASHMIR"
}
SouthList contains following list['TAMIL NADU']
{
	"parent_folder" : "801-Aby22GXjs7",
	"name" : "TAMIL NADU"
}
NorthList contains following list['JAMMU KASHMIR', 'HARYANA']
{
	"parent_folder" : "801-wud0JvV70D",
	"name" : "HARYANA"
}
NorthList contains following list['JAMMU KASHMIR', 'HARYANA', 'RAJASTHAN']
{
	"parent_folder" : "801-wud0JvV70D",
	"name" : "RAJASTHAN"
}
NorthList contains following list['JAMMU KASHMIR', 'HARYANA', 'RAJASTHAN', 'UP WEST']
{
	"parent_folder" : "801-wud0JvV70D",
	"name" : "UP WEST"
}
NorthList contains following list['JAMMU KASHMIR', 'HARYANA', 'RAJASTHAN', 'UP WEST', 'UTTARAKHAND']
{
	"parent_folder" : "801-wud0JvV70D",
	"name" : "UTTARAKHAND"
}
SouthList contains following list['TAMIL NADU', 'ANDHRA PRADESH']
{
	"parent_folder" : "801-Aby22GXjs7",
	"name" : "ANDHRA PRADESH"
}
SouthList contains following list['TAMIL NADU', 'ANDHRA PRADES

In [36]:
print(len(north_folder))
print(len(south_folder))
north_id = []
south_id = []
for x in range(len(north_folder)):
  north_id.append(north_folder[x]['result']['id'])
for x in range(len(south_folder)):
  south_id.append(south_folder[x]['result']['id'])

print(north_id)
print(south_id)

6
3
['801-TliYIdSFJ7', '801-p8QMTywVJG', '801-ssBSoIMOUh', '801-pAtXjOPxFH', '801-tMtlGMxHsc', '801-RYOSM5y6OR']
['801-VxAeAHciml', '801-nE72QWPPjz', '801-4yS4swq6vJ']


In [40]:
def create_town(townvalue,state_id,token):
      #cellvalue1 = list(cellvalue)
      #if cellvalue not in cellvalue1:
        url = "https://api.cloud.jiosurveillance.com/folders?op=CREATE"
        payload = '{\n\t\"parent_folder\" : \"' + state_id + '\",\n\t\"name\" : \"' + townvalue + "\"\n}"
        print(payload)
        headers = {
          'Content-Type': 'application/json',
          'Authorization': 'Bearer %s' % token
        }

        response = requests.request("POST", url, headers=headers, data=payload, timeout=10)
        r = response.json()
        return r
        #cellvalue1.add(cellvalue)


In [41]:
#print(state_folder)
#state_id = state_folder['result']['id']
#print(state_id)
for sh in xlrd.open_workbook('/content/Tree structure.xlsx').sheets():
  northtownList = []
  southtownlist = []
  townn_folder=[]
  towns_folder = []
  i=0
  for row in range(1, sh.nrows):
    for col in range(0,9):
      mycell = sh.cell(row,col).value
      for x in range(len(northList)):
        if mycell == northList[x]:
          town = sh.cell(row, col+1).value
        #statecell.value = str(statecell.value)
          if town.lower() in [name.lower() for name in northtownList]:
            break
          else:
            northtownList.append(town)
            #print("NorthtownList contains following list" + str(northtownList))
            townn_folder.append(create_town(town,north_id[x],token))
      for x in range(len(southList)):
        if mycell == southList[x]:
          town = sh.cell(row,col+1).value
          if town.lower() in [name.lower() for name in southtownlist]:
            break
          else:
            southtownlist.append(town)
            towns_folder.append(create_town(town,south_id[x],token))


{
	"parent_folder" : "801-TliYIdSFJ7",
	"name" : "Jammu"
}
{
	"parent_folder" : "801-VxAeAHciml",
	"name" : "Trichy"
}
{
	"parent_folder" : "801-p8QMTywVJG",
	"name" : "Panipat"
}
{
	"parent_folder" : "801-ssBSoIMOUh",
	"name" : "Sirohi"
}
{
	"parent_folder" : "801-pAtXjOPxFH",
	"name" : "Meerut"
}
{
	"parent_folder" : "801-VxAeAHciml",
	"name" : "Pondicherry"
}
{
	"parent_folder" : "801-pAtXjOPxFH",
	"name" : "Muzaffarnagar"
}
{
	"parent_folder" : "801-pAtXjOPxFH",
	"name" : "Aligarh"
}
{
	"parent_folder" : "801-pAtXjOPxFH",
	"name" : "Hapur"
}
{
	"parent_folder" : "801-pAtXjOPxFH",
	"name" : "Bijnore"
}
{
	"parent_folder" : "801-pAtXjOPxFH",
	"name" : "Moradabad"
}
{
	"parent_folder" : "801-pAtXjOPxFH",
	"name" : "Rai Bareily"
}
{
	"parent_folder" : "801-pAtXjOPxFH",
	"name" : "Bareilly"
}
{
	"parent_folder" : "801-tMtlGMxHsc",
	"name" : "Dehradun"
}
{
	"parent_folder" : "801-nE72QWPPjz",
	"name" : "Tirupati"
}
{
	"parent_folder" : "801-VxAeAHciml",
	"name" : "Tiripur"
}
{
	"parent_f

In [47]:
print(townn_folder)
print(towns_folder)
print(len(northtownList))
print(len(southtownlist))

[{'success': True, 'result': {'id': '801-6aVWVApX7K', 'name': 'Jammu', 'subfolders': [], 'cameras': [], 'owner_id': '801000011026', 'owner_name': 'rr.demo@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}, {'success': True, 'result': {'id': '801-Kcm6UY2RHN', 'name': 'Panipat', 'subfolders': [], 'cameras': [], 'owner_id': '801000011026', 'owner_name': 'rr.demo@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}, {'success': True, 'result': {'id': '801-Hi0UYFtOWc', 'name': 'Sirohi', 'subfolders': [], 'cameras': [], 'owner_id': '801000011026', 'owner_name': 'rr.demo@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}, {'success': True, 'result': {'id': '801-qrExA9n2DF', 'name': 'Meerut', 'subfolders': [], 'camera

In [51]:
#print(town_folder[1])
townn_id = []
towns_id = []
print(len(townn_folder))
for x in range(len(townn_folder)):
  #try:
    #print(x,town_folder[x]['result']['id'])
    townn_id.append(townn_folder[x]['result']['id'])
  #except KeyError as error:
    #print(x,error)
    #town_id.append("801-z6FtD3n7nh")
  #town_id.append(town_folder[x]['result']['id'])
  #print(town_folder[x]['success']['id'])
#print(type(town_id[14]))
#print(len(town_id))
for x in range(len(towns_folder)):
      towns_id.append(towns_folder[x]['result']['id'])



print(len(townn_id))
print(len(towns_id))





24
24
7


In [53]:
def create_jp(jpvalue,town_id,token):
      #cellvalue1 = list(cellvalue)
      #if cellvalue not in cellvalue1:
        url = "https://api.cloud.jiosurveillance.com/folders?op=CREATE"
        payload = '{\n\t\"parent_folder\" : \"' + town_id + '\",\n\t\"name\" : \"' + jpvalue + "\"\n}"
        print(payload)
        headers = {
          'Content-Type': 'application/json',
          'Authorization': 'Bearer %s' % token
        }

        response = requests.request("POST", url, headers=headers, data=payload, timeout=10)
        r = response.json()
        return r
        #cellvalue1.add(cellvalue)

In [54]:
#jplist = []
for sh in xlrd.open_workbook('/content/Tree structure.xlsx').sheets():
  jplist = []
  jslist = []
  jp_folder = []
  js_folder = []
  #print(northtownList)
  for row in range(1, sh.nrows):
    for col in range(1, 11):
      mycell = sh.cell(row,col).value
      #print(mycell)
      for x in range(len(northtownList)):
        #print(x)
        if mycell ==northtownList[x]:
          #print(mycell,x)
          jp = sh.cell(row,col+2).value
          #print('jp',jp)
          if jp.lower() in [name.lower() for name in jplist]:
            break
          else:
            jplist.append(jp)
            jp_folder.append(create_jp(jp,townn_id[x],token))
      for x in range(len(southtownlist)):
        if mycell == southtownlist[x]:
          jp = sh.cell(row,col+2).value
          if jp.lower() in [name.lower() for name in jslist]:
            break
          else:
            jslist.append(jp)
            js_folder.append(create_jp(jp,towns_id[x],token))

      


{
	"parent_folder" : "801-6aVWVApX7K",
	"name" : "I-JK-RESI-ENB-0005"
}
{
	"parent_folder" : "801-jYBed2Lr8b",
	"name" : "I-TN-LLGD-ENB-9009"
}
{
	"parent_folder" : "801-Kcm6UY2RHN",
	"name" : "I-HR-GHAN-ENB-9019"
}
{
	"parent_folder" : "801-Hi0UYFtOWc",
	"name" : "I-RJ-PNDW-ENB-9015"
}
{
	"parent_folder" : "801-Hi0UYFtOWc",
	"name" : "I-RJ-PNDW-ENB-9023"
}
{
	"parent_folder" : "801-qrExA9n2DF",
	"name" : "I-UW-MWNA-ENB-9021"
}
{
	"parent_folder" : "801-xYzx7jp7B1",
	"name" : "I-TN-VPPM-ENB-0026"
}
{
	"parent_folder" : "801-ZuAhqQiRPK",
	"name" : "I-UW-MZGR-ENB-9034"
}
{
	"parent_folder" : "801-ZuAhqQiRPK",
	"name" : "I-UW-KHTL-ENB-9003"
}
{
	"parent_folder" : "801-ZuAhqQiRPK",
	"name" : "I-UW-MZGR-ENB-9032"
}
{
	"parent_folder" : "801-ZuAhqQiRPK",
	"name" : "I-UW-KHTL-ENB-9001"
}
{
	"parent_folder" : "801-ZuAhqQiRPK",
	"name" : "I-UW-PRQZ-ENB-9001"
}
{
	"parent_folder" : "801-qrExA9n2DF",
	"name" : "I-UW-MWNA-ENB-9019"
}
{
	"parent_folder" : "801-ZuAhqQiRPK",
	"name" : "I-UW-DBND-ENB-

In [56]:
jp_idlist = []
js_idlist = []
for x in range(len(jp_folder)):
   jp_idlist.append(jp_folder[x]['result']['id'])
   #print(jp_folder[x]['result']['id'])
   #i=0
print(len(jp_idlist))
for x in range(len(js_folder)):
   js_idlist.append(js_folder[x]['result']['id'])
print(len(js_idlist))


146
7


In [57]:
def create_city(cityvalue,jp_id,token):
      #cellvalue1 = list(cellvalue)
      #if cellvalue not in cellvalue1:
        url = "https://api.cloud.jiosurveillance.com/folders?op=CREATE"
        payload = '{\n\t\"parent_folder\" : \"' + jp_id + '\",\n\t\"name\" : \"' + cityvalue + "\"\n}"
        print(payload)
        headers = {
          'Content-Type': 'application/json',
          'Authorization': 'Bearer %s' % token
        }

        response = requests.request("POST", url, headers=headers, data=payload, timeout=10)
        r = response.json()
        return r
        #cellvalue1.add(cellvalue)

In [59]:
for sh in xlrd.open_workbook('/content/Tree structure.xlsx').sheets():
  ipnorth = []
  ipsouth=[]
  ipn_folder = []
  ips_folder = []
  #print(northtownList)
  for row in range(1, sh.nrows):
    for col in range(1, 11):
      mycell = sh.cell(row,col).value
      #print(mycell)
      for x in range(len(jplist)):
        #print(x)
        if mycell ==jplist[x] :
          #print(mycell,x)
          ip = sh.cell(row,col-9).value
          #print('jp',jp)
          if ip.lower() in [name.lower() for name in ipnorth]:
            break
          else:
            ipnorth.append(ip)
            #for x in jp_idlist:
            ipn_folder.append(create_city(ip,jp_idlist[x],token))
      for x in range(len(jslist)):
        if mycell == jslist[x]:
          ip = sh.cell(row,col-9).value
          #print(mycell,ip)
          if ip.lower() in [name.lower() for name in ipsouth]:
            break
          else:
            ipsouth.append(ip)
            ips_folder.append(create_city(ip,js_idlist[x],token))

      
            
#print(jpcity)

{
	"parent_folder" : "801-1jjE0M24Ic",
	"name" : "10.213.234.197"
}
{
	"parent_folder" : "801-7eiLDdbGZQ",
	"name" : "10.218.238.197"
}
{
	"parent_folder" : "801-pCKEvr3zG4",
	"name" : "10.213.37.213"
}
{
	"parent_folder" : "801-ql16AjhM7n",
	"name" : "10.211.3.45"
}
{
	"parent_folder" : "801-vkDNurQ719",
	"name" : "10.211.159.69"
}
{
	"parent_folder" : "801-SdlnJGkpVt",
	"name" : "10.215.165.77"
}
{
	"parent_folder" : "801-ajD5swzKBv",
	"name" : "10.218.199.110"
}
{
	"parent_folder" : "801-XwBPD4HlC7",
	"name" : "10.215.136.69"
}
{
	"parent_folder" : "801-yGVa6IcIFw",
	"name" : "10.215.135.253"
}
{
	"parent_folder" : "801-aHXgLAyseH",
	"name" : "10.215.136.61"
}
{
	"parent_folder" : "801-meyh0AUXuG",
	"name" : "10.215.139.45"
}
{
	"parent_folder" : "801-PkFNXXn01B",
	"name" : "10.215.222.53"
}
{
	"parent_folder" : "801-jf6Zagied3",
	"name" : "10.214.249.173"
}
{
	"parent_folder" : "801-F3C7YFjNJ2",
	"name" : "10.214.243.133"
}
{
	"parent_folder" : "801-1CiLcxW2bD",
	"name" : "10.214.2

In [ ]:
#print(city_folder)
city_idlist = []
for x in range(len(city_folder)):
   city_idlist.append(city_folder[x]['result']['id'])
   #print(jp_folder[x]['result']['id'])
print(city_idlist)

['801-pVgM2xqn31', '801-GTn51FbSHL', '801-9JLDN1MQ4H', '801-hklvXQG0ek', '801-djDTEMx84M', '801-uR3EDAWND0', '801-pk3OU2lAob', '801-S36xkewc4b', '801-mTyfTEpRKe', '801-cHUpLCqy93', '801-63JVmu0tLi', '801-OBdOUBqt3h', '801-MqqqxMEwB8', '801-lK8DZmOlSQ', '801-D8KUqxDvtI', '801-TUFr8ggRdH', '801-FAT5ficThH', '801-wfg65wgKbx', '801-E12nvir4Oc', '801-wAD5YhECFg', '801-HRcO4cg1QM', '801-BZWhMXDsrB', '801-msdthMGjyj', '801-FtXVee7k9j', '801-2uxvoLujhH', '801-ZjpEVFm91w', '801-WJKR5JHdnL', '801-ezu5xas5v6', '801-xXzMOgUhxk', '801-t1M8wQRRXo', '801-g1JyhDj281', '801-BRe4Mo6i2i', '801-9Uo3onbh3T', '801-zAX1aZQbik', '801-qC6QKf6Qxx', '801-0a6xtUFGsi', '801-CJoHqeaQao', '801-N44644HHcL', '801-ldhECz0ruq', '801-aRaqOkAbfU', '801-LFGoegeDkS', '801-osQi3bRLuJ', '801-RBoH7x0oej', '801-7iWAvRblb5', '801-zlfuzmTq1r', '801-fwcsW9MuDq', '801-AuuF025fgq', '801-Ca7o7OJrd2', '801-eEhJjKPbxP', '801-VFvolmFblg', '801-BxXj8eoAOI', '801-DVdLfM17kd', '801-pKgiO5bjTm', '801-X7yVpQfdGX', '801-YScK09ZtD8', '801-rpho

In [ ]:
def create_mail(mailvalue,city_id,token):
      #cellvalue1 = list(cellvalue)
      #if cellvalue not in cellvalue1:
        url = "https://api.cloud.jiosurveillance.com/folders?op=CREATE"
        payload = '{\n\t\"parent_folder\" : \"' + city_id + '\",\n\t\"name\" : \"' + mailvalue + "\"\n}"
        print(payload)
        headers = {
          'Content-Type': 'application/json',
          'Authorization': 'Bearer %s' % token
        }

        response = requests.request("POST", url, headers=headers, data=payload, timeout=10)
        r = response.json()
        return r
        #cellvalue1.add(cellvalue)

In [ ]:
for sh in xlrd.open_workbook('/content/Tree structure.xlsx').sheets():
  jpmail = []
  i=0
  mail_folder = []
  #print(northtownList)
  for row in range(1, sh.nrows):
    for col in range(1, 12):
      mycell = sh.cell(row,col).value
      #print(mycell)
      for x in jpcity:
        #print(x)
        if mycell ==x :
          #print(mycell,x)
          mail = sh.cell(row,col+10).value
          #print('jp',jp)
          if mail.lower() in [name.lower() for name in jpmail]:
            break
          else:
            jpmail.append(mail)
            mail_folder.append(create_mail(mail,city_idlist[i],token))
            i+=1


{
	"parent_folder" : "801-pVgM2xqn31",
	"name" : "INJKRESIRESITW0004ENBCAM001"
}
{
	"parent_folder" : "801-GTn51FbSHL",
	"name" : "INHRGHANXXXXTW0001ENBCAM001"
}
{
	"parent_folder" : "801-9JLDN1MQ4H",
	"name" : "INRJPNDWXXXXXXXXXXENBCAM001"
}
{
	"parent_folder" : "801-hklvXQG0ek",
	"name" : "INRJPNDWXXXXTW0013ENBCAM001"
}
{
	"parent_folder" : "801-djDTEMx84M",
	"name" : "INUWMWNAXXXXTW0003ENBCAM001"
}
{
	"parent_folder" : "801-uR3EDAWND0",
	"name" : "INUWBALCBMHITW0001ENBCAM001"
}
{
	"parent_folder" : "801-pk3OU2lAob",
	"name" : "INUWKHTLXXXXTW0004ENBCAM001"
}
{
	"parent_folder" : "801-S36xkewc4b",
	"name" : "INUWMZGRXXXXTW0005ENBCAM001"
}
{
	"parent_folder" : "801-mTyfTEpRKe",
	"name" : "INUWKHTLXXXXTW0003ENBCAM001"
}
{
	"parent_folder" : "801-cHUpLCqy93",
	"name" : "INUWPRQZPRQZTW0001ENBCAM001"
}
{
	"parent_folder" : "801-63JVmu0tLi",
	"name" : "INUWMWNAXXXXTW0001ENBCAM001"
}
{
	"parent_folder" : "801-OBdOUBqt3h",
	"name" : "INUWDBNDXXXXTW0001ENBCAM001"
}
{
	"parent_folder" : "801-Mq

In [ ]:
print(mail_folder)

[{'success': True, 'result': {'id': '801-sbFGXSXSob', 'name': 'INJKRESIRESITW0004ENBCAM001', 'subfolders': [], 'cameras': [], 'owner_id': '801000012219', 'owner_name': 'rr.test4@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}, {'success': True, 'result': {'id': '801-QdbYEB3mdG', 'name': 'INHRGHANXXXXTW0001ENBCAM001', 'subfolders': [], 'cameras': [], 'owner_id': '801000012219', 'owner_name': 'rr.test4@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}, {'success': True, 'result': {'id': '801-suahWKUYYr', 'name': 'INRJPNDWXXXXXXXXXXENBCAM001', 'subfolders': [], 'cameras': [], 'owner_id': '801000012219', 'owner_name': 'rr.test4@gmail.com', 'root': False, 'permissions': ['admin', 'live', 'archive', 'ptz', 'events', 'faces_read', 'faces_add', 'faces_admin'], 'geo': None}}, {'success': True, 'result': {'